In [2]:
import sys
sys.path.append('..')

import torch
import pandas as pd
import re
import numpy as np
from datasets import DatasetDict, Dataset
from torch import nn 
from transformers import BertModel, AutoModelForSequenceClassification, BertForTokenClassification, EarlyStoppingCallback

from utils import *
from dataset import *
from preprocess import *
from wrapper import *

torch.cuda.is_available()

True

In [3]:
train_df_full = pd.read_csv('../data/train.csv', sep='\t')
test_df_full = pd.read_csv('../data/test.csv', sep='\t')

train_df = train_df_full[:1000]
test_df = test_df_full.iloc[np.random.randint(0, len(test_df_full), 200)]

(train_df.label == 0).sum(), (train_df.label == 1).sum()  # label distribution is pretty balanced

(257, 743)

In [7]:
folds = easy_ensenble_generate_kfolds(1000, 5, minority_idx)

MODEL_NAME = 'hfl/chinese-macbert-base'

# Define training arguments
arguments = MyTrainingArguments(
    output_dir="sample_trainer",  # output directory
    per_device_train_batch_size=16,  # set training and eval batch size
    per_device_eval_batch_size=16,
    num_train_epochs=4,  # number of training epochs
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",  # save checkpoint at each epoch
    learning_rate=1e-5,
    load_best_model_at_end=True,
    label_names=['labels'],   # need to specify this to pass the labels to the trainer
    epsilon=0, 
    gamma=0.2,
    alpha=0.3,
)

test = DatasetWithAuxiliaryEmbeddings(
    test_df, 
    model_name=MODEL_NAME, 
    aux_model_name='uer/roberta-base-finetuned-cluener2020-chinese',
    test=True, split_words=False, 
)
test.tokenize()
test.construct_dataset()

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


In [10]:
k = 10
minority_idx = np.array(train_df.index[train_df.label == 0].tolist())
folds = easy_ensenble_generate_kfolds(L=len(train_df), k=k, minority_idx=minority_idx)
logits = []

val_idx = folds[0]

train = DatasetWithAuxiliaryEmbeddings(train_df, model_name=MODEL_NAME, 
    aux_model_name='uer/roberta-base-finetuned-cluener2020-chinese', 
    train_val_split=0.8, split_words=False
)
train.tokenize()
train.construct_dataset(val_idx=val_idx)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()
d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()
